# TEI Conference 2019 #

## Etape 3 : L'Analyse ##

### Préambule ###

L'objectif présent est d'analyser les abstracts afin de les regrouper en cluster, permettant de dégager des thématiques, de comparer les différentes conférences ou bien d'étudier si les distinctions faites par les organisateurs de la conférence 2019 entre les différentes interventions est bien pertinente.
Nous pourrons également critiquer les choix de l'ordinateur.

Nous allons faire ici de la classification non supervisée grâce au package SKLearn, librairie dédiée au Machine Learning notamment.

### Les packages ### 

Il faut lancer la cellule ci-dessous une seule fois afin de télécharger la librairie sklearn. Cela doit être fait une seule fois, au premier lancement. Ensuite, il ne faudra plus jamais le lancer. Il est possible qu'il faille redémarrer le noyau après.

In [ ]:
pip install sklearn
pip install nltk
pip install pandas
pip install gensim

In [ ]:
pip install spacy

In [ ]:
pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-2.2.0/en_core_web_sm-2.2.0.tar.gz

Il faut lancer la cellule ci-dessous à chaque lancement de ce notebook-ci.

In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import nltk
from nltk.stem import PorterStemmer
import os
import spacy
import csv
import pandas as pd
import numpy as np
import scipy as sp
import sklearn
import sys
from nltk.corpus import stopwords
from gensim.models import ldamodel
import gensim.corpora
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.decomposition import NMF
from sklearn.preprocessing import normalize


D'abord, on crée le répertoire dans le cache du contenu qui recevra les textes lemmatisés :

In [2]:
Path = "./cache2019/cacheTXT/" #chemin permettant d'accès aux abstracts
Path_output = "./cache2019/cacheLEM/" #chemin de sortie des abstracts stemmés
filelist = os.listdir(Path) #filelist est une liste regroupant tous les chemins vers les différents abstracts.
if not os.path.exists("./cache2019/cacheLEM/"): #permet de créer un dossier dans le cache s'il est supprimé.
    os.makedirs("./cache2019/cacheLEM/")

#### Méthode 1 : tokenisation et lemmatisation ####

Je propose ci-dessous de lemmatizer le texte avec spaCy. Ma lemmatisation va retourner une forme canonique de chaque mot. La tokenisation et la lemmatisation s'appuient sur un petit corpus que j'ai importé (en_core_web_sm) depuis la base de spaCy. En entrée se trouve le texte normalisé de chaque abstract, et en sortie nous retrouvons ce même texte, mais où chaque mot est remplacé par son lem.

In [3]:
for abstract in filelist:
    with open(Path + abstract, "r", encoding="UTF-8") as y:
        texte = y.read()
        nlp = spacy.load("en_core_web_sm") #nlp est un corpus de test permettant de tokenizer efficacement l'anglais à partir d'un petit corpus (j'ai choisi le petit corpus, pas le gros)
        doc = nlp(texte) #doc est le texte annoté. Ce n'est pas une str pour autant
        liste_mots_lemmatise = [] # j'instancie une liste qui accueillera chaque lemme (le lemme étant une str)
        for token in doc:
            liste_mots_lemmatise.append(token.lemma_) #token.lemma_ est une str. Cette méthode tokenise puis lematise. Il crée une liste qu'on va joindre ensuite.
        resultat = ' '.join(liste_mots_lemmatise)
        with open(Path_output + abstract, "w", encoding="UTF-8") as z:
            z.write(resultat) #resultat est une str
         

#je peux demander de l'aide pour comprendre une doc
#pour demain 9h du matin, il faut que j'écrive une définition de tokenisation, lemmatisation et POS taging, leur impact et leur importance. Demain à 11h (9h métropole) je le montre à Clérice
#Pour lundi 9h, me farcir les polycopiés (page 31 à 38 du polycop à 38 pages). Il faut aussi lire l'article que Clérice m'a envoyé.
#Regarder ce qu'est le topic modeling (en non supervisé), essayez de trouver un tuto là dessus.

#### Méthode 2 : tokenization + stemmisation ####

Dans cette seconde méthode, le texte est tokenisé (grâce à spaCy toujours) mais il est stemmé et non pas lemmatisé. Le stemming consiste à retirer les terminaisons pour garder une forme canonique de chaque mot. Le fonctionnement est identique à celui de la lemmatisation, pour ma démarche. Le résultat est également stocké dans un cache.

In [ ]:
Path = "./cache2019/cacheTXT/" #chemin permettant d'accès aux abstracts
Path_output = "./cache2019/cacheSTEM/" #chemin de sortie des abstracts stemmés
filelist = os.listdir(Path) #filelist est une liste regroupant tous les chemins vers les différents abstracts.
if not os.path.exists("./cache2019/cacheSTEM/"): #permet de créer un dossier dans le cache s'il est supprimé.
    os.makedirs("./cache2019/cacheSTEM/")

In [ ]:
stemmer = PorterStemmer() #J'importe une méthode acceptant une chaîne de caractère à une variable.
for abstract in filelist:
    with open(Path + abstract, "r", encoding="UTF-8") as y:
        texte = y.read()
        liste_mots_tokenise = [] # j'instancie une liste qui accueillera chaque lemme (le lemme étant une str)
        liste_mots_a_tokenise = texte.split() #je crée une liste sur laquelle je vais pouvoir boucler
        for elem in liste_mots_a_tokenise:
            mot_tokenise = stemmer.stem(elem) #mot_tokenise est une str, et est le lemme du mot sur lequel je boucle
            liste_mots_tokenise.append(mot_tokenise) #j'ajoute chaque mot à une liste, qu'ensuite je join pour recréer l'abstract sous la forme d'une str
        resultat = ' '.join(liste_mots_tokenise)
        with open(Path_output + abstract, "w", encoding="UTF-8") as z:
            z.write(resultat) #resultat est une str

Ensuite, il faut choisir quel set sera loadé et analysé entre les abstracts lemmatisés et les abstracts loadés.

In [4]:
#Pour utiliser les textes lemmatisés, c'est ici

documents = []
Path = "./cache2019/cacheLEM/"
filelist = os.listdir(Path) #filelist est une liste regroupant tous les chemins vers les différents abstracts.

for abstract in filelist:
    with open(Path + abstract, "r", encoding="UTF-8") as y:
        texte = y.read()
        documents.append(texte)

In [ ]:
#Pour utiliser les textes stemmés, c'est ici

documents = []
Path = "./cache2019/cacheSTEM/"
filelist = os.listdir(Path) #filelist est une liste regroupant tous les chemins vers les différents abstracts.

for abstract in filelist:
    with open(Path + abstract, "r", encoding="UTF-8") as y:
        texte = y.read()
        documents.append(texte)





## Définir les clusters ##

Après avoir tokennisé et lemmatisé/stemmé nos abstracts, nous allons maintenant les vectoriser (représenter chaque mot sous la forme d'un vecteur sur un plan ordonné en deux dimensions) puis ensuite nous allons laisser la machine définir des clusters, c'est à dire des groupes de mots qui ont un sens commun.

Ici, on associe à chaque mot de tous les abstracts une coordonnée unique à chaque mot utilisé, pour ensuite pouvoir les placer sur un plan en 2D et ainsi relever des ressemblances.

In [5]:
#vectorize the text i.e. convert the strings to numeric features
vectorizer = TfidfVectorizer(stop_words='english')
X = vectorizer.fit_transform(documents)

Ensuite, on réunit les termes vectorisés en cluster, c'est à dire en groupe de ressemblance.

In [6]:
#cluster documents, ici 8 clusters
true_k = 8
model = KMeans(n_clusters=true_k, init='k-means++', max_iter=100, n_init=1)
model.fit(X)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=100,
       n_clusters=8, n_init=1, n_jobs=None, precompute_distances='auto',
       random_state=None, tol=0.0001, verbose=0)

Enfin, on montre les termes que l'ordinateur a pu rassembler ensemble.

In [7]:
#print top terms per cluster clusters
print("Top terms per cluster:")
order_centroids = model.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names()
for i in range(true_k):
    print("Cluster %d:" % i)
    for ind in order_centroids[i, :10]:
        print(' %s' % terms[ind])
        

        


Top terms per cluster:
Cluster 0:
 token
 hi
 tokenized
 tei
 tokenize
 break
 pb
 unary
 searchable
 cwb
Cluster 1:
 correspondence
 bibliographic
 cmif
 letter
 tei
 correspsearch
 hallernet
 metadata
 pron
 jtei
Cluster 2:
 pron
 text
 tei
 use
 paper
 encode
 corpus
 element
 odd
 edition
Cluster 3:
 austria
 bavarian
 dialect
 bowers
 creation
 breuer
 stöckle
 dictionary
 article
 tei
Cluster 4:
 kosh
 dictionary
 api
 access
 index
 datum
 data
 file
 xml
 path
Cluster 5:
 pron
 tei
 line
 project
 rhyme
 poem
 use
 digital
 xml
 build
Cluster 6:
 sales
 catalogues
 structuring
 automatic
 scale
 manuscript
 打球場
 entities
 entire
 entirely
Cluster 7:
 punctuation
 sign
 pron
 manuscript
 medieval
 interpretative
 modern
 transformation
 edition
 transcription


## Cluster de documents ##

Grâce à un système de prédiction découlant de l'apprentissage et de l'entraînement sur les termes des abstracts, nous pouvons classifier les abstracts en 8 catégories. Nous pourrons ensuite les comparer avec le classement déjà fait par les organisateurs de la conférence TEI 2019.

In [11]:
Path = "./cache2019/cacheLEM/" #On peut remplacer le cache STEM par LEM à la place, pour ainsi comparer les deux sorties.
filelist = os.listdir(Path) #filelist est une liste regroupant tous les chemins vers les différents abstracts.
liste_triee = [] #c'est la liste contenant les infos triées de tous les abstracts


for abstract in filelist:
    reference = abstract.replace('.txt', '') #je normalise le nom du texte pour le faire correpondre avec celui indiqué dans le tableau csv réunissant toutes les informations
    with open(Path + abstract, "r", encoding="UTF-8") as y:
        liste_resultat_unitaire = [] #j'instancie la liste des infos propres à chaque abstract
        texte = y.read()
        X = vectorizer.transform([texte]) #je vectorise le texte de l'abstract
        predicted = model.predict(X)  #j'attribue l'abstract à un cluster. predicted est de la classe numpy array : c'est un vecteur
        with open('./cache2019/TEI2019.csv', 'r') as csvfile:
            read = csv.reader(csvfile, delimiter = ',')
            titre = ''
            auteurs = ''
            for row in read:
                if row[4] == reference:
                    titre = row[2]
                    auteurs = row[0]
        liste_resultat_unitaire.append(titre)
        liste_resultat_unitaire.append(auteurs)
        liste_resultat_unitaire.append(int(predicted))
        liste_triee.append(liste_resultat_unitaire)
        

print("Dans le groupe 1, il y a :")
for elem in liste_triee:
    if elem[2] == 0:
        print("\t", elem[0] )
        
print("\n Dans le groupe 2, il y a :")
for elem in liste_triee:
    if elem[2] == 1:
        print("\t", elem[0] )
        
print("\n Dans le groupe 3, il y a :")
for elem in liste_triee:
    if elem[2] == 2:
        print("\t", elem[0] )

print("\n Dans le groupe 4, il y a :")
for elem in liste_triee:
    if elem[2] == 3:
        print("\t", elem[0] )

print("\n Dans le groupe 5, il y a :")
for elem in liste_triee:
    if elem[2] == 4:
        print("\t", elem[0] )
        
print("\n Dans le groupe 6, il y a :")
for elem in liste_triee:
    if elem[2] == 5:
        print("\t", elem[0] )
        
print("\n Dans le groupe 7, il y a :")
for elem in liste_triee:
    if elem[2] == 6:
        print("\t", elem[0] )
        
print("\n Dans le groupe 8, il y a :")
for elem in liste_triee:
    if elem[2] == 7:
        print("\t", elem[0] )




Dans le groupe 1, il y a :
	 Advantages and challenges of tokenized TEI

 Dans le groupe 2, il y a :
	 correspSearch v2 – New ways of exploring correspondence
	 Modeling FRBR Entities and their Relationships with TEI: A Look at HallerNet Bibliographic Descriptions
	 Referencing annotations as a core concept of the hallerNet edition and research platform
	 TEI encoding of correspondence: A community effort

 Dans le groupe 3, il y a :
	 Using Machine Learning for the Automated Classification of Stage Directions in TEI-Encoded Drama Corpora
	 TEI XML and Delta Format Interchangeability
	 Introducing Objectification: when is an <object> a <place>?
	 Analyzing and Visualizing Uncertain Knowledge: Introducing the PROVIDEDH Open Science Platform
	 Referencing an editorial ontology from the TEI: An attempt to overcome informal typologies
	 Case Study TEI Customization: A Restricted TEI Format for Edition Open Access (EOA)
	 In search of comity: TEI for distant reading
	 Reconceiving TEI model

Voici le plan et l'agencement des conférences tel qu'il est indiqué ici : https://graz-2019.tei-c.org/wp-content/uploads/2019/09/ProgrammheftFINAL.pdf

TEI, formal ontologies,controlled vocabularies and Linked OpenData :
- Making Linkable Data from Account Books: Bookkeeping Ontology in the Digital Edition Publishing Cooperative for Historical Accounts
- Inscriptions, Hieroglyphs, Linguistics and Beyond! The Corpus of Classic Mayan as an Ontological Information Resource
- Modeling FRBR Entities and their Relationships with TEI: A Look at HallerNet Bibliographic Descriptions
- Referencing an editorial ontology from the TEI: An attempt to overcome informal typologies
- Text Graph Ontology. A Semantic Web approach to represent genetic scholarly editions

TEI and models of text :
- A realistic theory of textuality and its consequences on digital text representation
- Genesis and Variance: From Letter to Literature
- Between freedom and formalisation: a hypergraph model for representing the nature of text
- Reflecting the Influence of Technology on Models of Text in Scholarly Digital Editing
- Introducing Objectification: when is an <object> a <place> ?
- An Encoding Strategic Proposal of “Ruby” Texts: Examples from Japanese Texts
- Referencing annotations as a core concept of the hallerNet edition and research platform
- Recreating history through events
- Document Modeling with the TEI Critical Apparatus
- Exploring TEI structures to find distinctive features of text types
- Reconceiving TEI models of theatrical performance text with reference to promptbooks
- What is a Line? Encoding and Counting Lines in Early Modern Dramatic Texts
    
TEI across corpora,languages, and cultures :
- Growing collections of TEI texts: Some lessons from SARIT
- Towards larger corpora of Indic texts: For now, minimize metatext
- Encoding history in TEI: A corpus-oriented approach for investigating Tibetan historiography
- Advantages and challenges of tokenized TEI
- A sign of the times: medieval punctuation, its encoding and its rendition in modern times

TEI annotation and publication :
- Analyzing and Visualizing Uncertain Knowledge: Introducing the PROVIDEDH Open Science Platform
- The Prefabricated Website: Who Needs a Server Anyway?
- correspSearch v2 –New ways of exploring correspondence
- Validating @selector: a regular expression adventure
- TEI encoding of correspondence: A community effort

TEI simplification and extension :
- Opportunities and challenges of the TEI for scholarly journals in the Humanities
- Archiving a TEI project FAIRly
- Creating high-quality print from TEI documents
- Native-TEI dialectal dictionary for Bavarian dialects in Austria: data structure, software and workflow
- An Attempt of Dissemination of TEI in a TEI-underdeveloped country: Activities of the SIG EAJ
- Refining the Current Teaching Methodology of the TEI through the Analysis of Server Logs
- Using Github and its Integrations to Create, Test, and Deploy a Digital Edition

TEI environments and infrastructures :
- Parla-CLARIN: TEI guidelines for corpora of parliamentary proceedings
- Challenges in encoding parliamentary data: between applause and interjections
- A TEI customization for the description of paper and watermarks
- How we tripled our encoding speed in the Digital Victorian Periodical Project
- Manuscripta-The editor from past to future
- Highlighting Our Examples: encoding XML examples in pedagogical contexts
- Case Study TEI Customization: A Restricted TEI Format for Edition Open Access (EOA)
- In search of comity: TEI for distant reading

TEI and beyond :interactions, interchange, integrations and interoperability :
- Using Machine Learning for the Automated Classification of Stage Directions in TEI-Encoded Drama Corpora
- TEI XML and Delta Format Interchangeability

TEI and non-XML technologies :
- Five Centuries of History in a Network
- Introducing an Open, Dynamic and Efficient Lexical Data Access for TEI-encoded Dictionaries on the Internet
- Getting Along with Relational Databases
- Using Microsoft Word for preparing XML TEI-compliant digital editions
- Scaling up Automatic Structuring of Manuscript Sales Catalogues

In [10]:
# ne pas le lancer, je stock ici l'exemple pour clusteriser des documents
for abstract in documents:
    X = vectorizer.transform([abstract])
    predicted = model.predict(X)

### Topic Modelling ###

Ensuite, je vais tenter de déterminer des sujets aux cluster, c'est à dire déterminer grâce aux mots de chaque cluster des thèmes propres à chaque cluster grâce à une étude de chacun des termes.

In [12]:
transformer = TfidfTransformer(smooth_idf=False)
x_tfidf = transformer.fit_transform(X)
xtfidf_norm = normalize(x_tfidf, norm='l1', axis=1)

/home/formation1/snap/jupyter/common/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:1466: RuntimeWarning: divide by zero encountered in true_divide
  idf = np.log(n_samples / df) + 1


In [13]:
num_topics = 8 #définition du nombre de topics
model = NMF(n_components=num_topics) 
model.fit(xtfidf_norm)

/home/formation1/snap/jupyter/common/lib/python3.7/site-packages/sklearn/decomposition/_nmf.py:109: RuntimeWarning: invalid value encountered in sqrt
  return np.sqrt(res * 2)


NMF(alpha=0.0, beta_loss='frobenius', init=None, l1_ratio=0.0, max_iter=200,
    n_components=8, random_state=None, shuffle=False, solver='cd', tol=0.0001,
    verbose=0)

In [14]:
def get_nmf_topics(model, n_top_words):
    
    #il faut obtenir le mot à partir de son vecteur identifiant pour afficher les termes de chaque sujet
    feat_names = vectorizer.get_feature_names()
    
    word_dict = {}
    for i in range(num_topics):
        
        #for each topic, obtain the largest values, and add the words they map to into the dictionary.
        words_ids = model.components_[i].argsort()[:-20 - 1:-1]
        words = [feat_names[key] for key in words_ids]
        word_dict['Topic # ' + '{:02d}'.format(i+1)] = words;
    
    return pd.DataFrame(word_dict)
get_nmf_topics(model, 20)

,Topic # 01,Topic # 02,Topic # 03,Topic # 04,Topic # 05,Topic # 06,Topic # 07,Topic # 08
0,stance,point,far,physical,theory,perception,facie,notational
1,artifact,notion,ask,advocate,1987,finally,theory,realist
2,notational,possibility,try,sameness,act,building,2009,property
3,theory,approach,physical,notion,punctuation,methodological,inline,pron
4,pron,interpret,study,notorious,notorious,notational,framework,text
5,goodman,recur,property,1987,hypertext,1968,2013b,far
6,representation,attribution,explanation,settle,recent,daniel,devise,constructivist
7,textuality,brief,fundamental,reading,mcgann,anti,fix,devise
8,debate,belief,language,brief,various,fix,2013a,rationale
9,property,nelson,notion,encode,decade,area,complex,notorious
